# Entregable 2
## Inteligencia Artificial

En este entregable se realizará la aplicación de Regresión Lineal y Regresión logística. 

## Regresión Lineal:

Trabajaremos con los datos de la clase de EDA [(Clase02)](https://github.com/andrescastro-itm/AI_ITM/blob/main/Clase02/EDA.ipynb). 

1. Deben cargar los datos y realizar una depuración de los mismos (pueden usar lo que se hizo en la clase de EDA)

2. Realizar una división del conjunto de entrenamiento en conjuntos de entrenamiento/prueba, con los porcentajes que consideren

3. Posteriormente, deben entrenar un regresor lineal y reportar el error cuadrático medio en la predicción del precio de venta _(SalePrice)_ en el conjunto de prueba

4. Realizar una remoción de algunas características (a discreción de ustedes) y realizar de nuevo el entrenamiento del regresor lineal. Reportar el error cuadrático medio en la predicción del precio de venta _(SalePrice)_ en el conjunto de prueba.

In [1]:
#celda para importar, pueden agregar las que requieran

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

Celda para la carga de datos:

In [2]:
df = pd.read_csv("train.csv",index_col=0)

In [3]:
df

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
Id,,,,,,,,,,,,,,,,,,,,,
1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1456,60,RL,62.0,7917,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,8,2007,WD,Normal,175000
1457,20,RL,85.0,13175,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,MnPrv,NaN,0,2,2010,WD,Normal,210000
1458,70,RL,66.0,9042,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,GdPrv,Shed,2500,5,2010,WD,Normal,266500


In [4]:
def clean_data(df):
    # Drop column: 'Alley'
    df = df.drop(columns=['Alley'])
    # Drop column: 'PoolQC'
    df = df.drop(columns=['PoolQC'])
    return df

df_clean = clean_data(df.copy())
df_clean.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,...,ScreenPorch,PoolArea,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
Id,,,,,,,,,,,,,,,,,,,,,
1,60,RL,65.0,8450,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,0,0,NaN,NaN,0,2,2008,WD,Normal,208500
2,20,RL,80.0,9600,Pave,Reg,Lvl,AllPub,FR2,Gtl,...,0,0,NaN,NaN,0,5,2007,WD,Normal,181500
3,60,RL,68.0,11250,Pave,IR1,Lvl,AllPub,Inside,Gtl,...,0,0,NaN,NaN,0,9,2008,WD,Normal,223500
4,70,RL,60.0,9550,Pave,IR1,Lvl,AllPub,Corner,Gtl,...,0,0,NaN,NaN,0,2,2006,WD,Abnorml,140000
5,60,RL,84.0,14260,Pave,IR1,Lvl,AllPub,FR2,Gtl,...,0,0,NaN,NaN,0,12,2008,WD,Normal,250000


## Solución a punto 1.

## Solución a punto 2.

## Solución a punto 3.

## Regresión Logística:

Estos puntos los deben trabajar con **alguno (o si prefieren, todos)** de los conjuntos de datos del proyecto final. 

Lo pueden realizar de dos formas:

- Seleccionar únicamente los datos correspondientes a las clases 1 y 2, es decir, los datos con etiqueta Sabana o Bosque. Verificar que les deben quedar en total 29540 datos entre los conjuntos de entrenamiento, validación y prueba. ó
- Trabajar con las tres clases aprovechando que sklearn utiliza el esquema uno-contra-todos (one-vs-rest OvR)

Realizar las siguientes tareas:

1. Entrenar un modelo de regresión logística con el conjunto de entrenamiento
2. Reportar los resultados logrados con el conjunto de validación en términos de precisión, recall y F1-score. Para este punto pueden usar [classification_report de sklearn](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html#sklearn.metrics.classification_report)
3. Mostrar una matriz de confusión para la clasificación hecha en el conjunto de validación


## Solución a punto 1.

## Solución a punto 2.

## Solución a punto 3.